In [ ]:
import codecs
from gensim import corpora
from collections import  defaultdict, deque
import numpy as np

In [ ]:
raw_text = ["During my second month of nursing school, our professor gave us a pop quiz", 
#             "I was a conscientious student and had breezed through the questions, until I read the last one:", 
# "What is the first name of the woman who cleans the school?",  "Surely this was some kind of joke.",
# "I had seen the cleaning woman several times. She was tall, dark-haired and in her 50s, but how would I know her name?"
]

#  """During my second month of nursing school, our professor gave us a pop quiz.  
# I was a conscientious student and had breezed through the questions, until I read the last one: 
# “What is the first name of the woman who cleans the school?”  Surely this was some kind of joke. 
# I had seen the cleaning woman several times. She was tall, dark-haired and in her 50s, but how would I know her name?  
# I handed in my paper, leaving the last question blank.  Before class ended, one student asked if the last question would count toward our quiz grade.  
# “Absolutely,” said the professor.  “In your careers you will meet many people. All are significant. They deserve your attention and care, 
# even if all you do is smile and say ‘hello’. I’ve never forgotten that lesson. I also learned her name was Dorothy."""

In [ ]:
class HuffmanNode:
    def __init__(self, word_id, frequency):
        self.word_id = word_id  
        self.frequency = frequency  
        self.left_child = None
        self.right_child = None
        self.father = None
        self.Huffman_code = []  
        self.path = []  


class HuffmanTree:
    def __init__(self, wordid_frequency_dict):
        self.word_count = len(wordid_frequency_dict)  
        self.wordid_code = {}
        self.wordid_path = {}
        self.root = None
        unmerge_node_list = [HuffmanNode(wordid, frequency) for wordid, frequency in
                             wordid_frequency_dict.items()]  
        self.huffman = [HuffmanNode(wordid, frequency) for wordid, frequency in
                        wordid_frequency_dict.items()]  
        #huffman tree
        self.build_tree(unmerge_node_list)
        #huffman code
        self.generate_huffman_code_and_path()

    def merge_node(self, node1, node2):
        sum_frequency = node1.frequency + node2.frequency
        mid_node_id = len(self.huffman)  
        father_node = HuffmanNode(mid_node_id, sum_frequency)
        if node1.frequency >= node2.frequency:
            father_node.left_child = node1
            father_node.right_child = node2
        else:
            father_node.left_child = node2
            father_node.right_child = node1
        self.huffman.append(father_node)
        return father_node

    def build_tree(self, node_list):
        while len(node_list) > 1:
            i1 = 0  
            i2 = 1  
            if node_list[i2].frequency < node_list[i1].frequency:
                [i1, i2] = [i2, i1]
            for i in range(2, len(node_list)):
                if node_list[i].frequency < node_list[i2].frequency:
                    i2 = i
                    if node_list[i2].frequency < node_list[i1].frequency:
                        [i1, i2] = [i2, i1]
            father_node = self.merge_node(node_list[i1], node_list[i2])  
            if i1 < i2:
                node_list.pop(i2)
                node_list.pop(i1)
            elif i1 > i2:
                node_list.pop(i1)
                node_list.pop(i2)
            else:
                raise RuntimeError('i1 should not be equal to i2')
            node_list.insert(0, father_node)  
        self.root = node_list[0]

    def generate_huffman_code_and_path(self):
        stack = [self.root]
        while len(stack) > 0:
            node = stack.pop()
            
            while node.left_child or node.right_child:
                code = node.Huffman_code
                path = node.path
                node.left_child.Huffman_code = code + [1]
                node.right_child.Huffman_code = code + [0]
                node.left_child.path = path + [node.word_id]
                node.right_child.path = path + [node.word_id]
                
                stack.append(node.right_child)
                node = node.left_child
            word_id = node.word_id
            word_code = node.Huffman_code
            word_path = node.path
            self.huffman[word_id].Huffman_code = word_code
            self.huffman[word_id].path = word_path
            
            self.wordid_code[word_id] = word_code
            self.wordid_path[word_id] = word_path

    
    def get_all_pos_and_neg_path(self):
        positive = []  
        negative = [] 
        for word_id in range(self.word_count):
            pos_id = []  
            neg_id = []  
            for i, code in enumerate(self.huffman[word_id].Huffman_code):
                if code == 1:
                    pos_id.append(self.huffman[word_id].path[i])
                else:
                    neg_id.append(self.huffman[word_id].path[i])
            positive.append(pos_id)
            negative.append(neg_id)
        return positive, negative

In [ ]:
def test_tree(word_frequency):
    print("word_frequency: ",word_frequency)
    tree = HuffmanTree(word_frequency)
    print('Word ID: ',tree.wordid_code)
    print('Word ID Path: ',tree.wordid_path)
    for i in range(len(word_frequency)):
        print('Word ID: ',tree.huffman[i].path)
    print('+ and - : ',tree.get_all_pos_and_neg_path())

In [ ]:
word_to_id = {w:i for i, w in enumerate(set(raw_text))}
id_to_word = {i:w for w, i in word_to_id.items()}
word_frequency = {i:raw_text.count(w) for w,i in word_to_id.items()}

In [ ]:
word_freq = {}
word_count_sum = 0
sentence_count = 0

for line in raw_text:
    line = line.strip().split(' ')
    word_count_sum += len(line)
    sentence_count += 1

    for word in line:
        try:
            word_freq[word] += 1
        except:
            word_freq[word] = 1
word_id = 0

id2word_dict = {}
word2id_dict = {}
wordId_frequency_dict = {}

min_count = 1
for per_word, per_count in word_freq.items():
    if per_count < min_count:  
        word_count_sum -= per_count
        continue
    id2word_dict[word_id] = per_word
    word2id_dict[per_word] = word_id
    wordId_frequency_dict[word_id] = per_count
    word_id += 1
word_count = len(word2id_dict)

# word_freq, word_count_sum, sentence_count
# word_count, id2word_dict, word2id_dict, wordId_frequency_dict

In [ ]:
class InputData:
    def __init__(self, raw_text, min_count):
       
        self.input_file = raw_text
        self.min_count = min_count  
        self.wordId_frequency_dict = {}  
        self.word_count = 0  
        self.word_count_sum = 0  
        self.sentence_count = 0  
        self.id2word_dict = {}  
        self.word2id_dict = {}  
        self._init_dict()  
        self.sample_table = []
        self._init_sample_table()  

        self.huffman_tree = HuffmanTree(self.wordId_frequency_dict)
        self.huffman_pos_path, self.huffman_neg_path = self.huffman_tree.get_all_pos_and_neg_path()

        self.word_pairs_queue = deque()
      
        print('Word Count is:', self.word_count)
        print('Word Count Sum is', self.word_count_sum)
        print('Sentence Count is:', self.sentence_count)
        print('Word Frequency:', self.wordId_frequency_dict)

    def _init_dict(self):
        word_freq = {}
        
        for line in self.input_file:
            line = line.strip().split(' ')  
            self.word_count_sum += len(line)
            self.sentence_count += 1
            for word in line:
                try:
                    word_freq[word] += 1
                except:
                    word_freq[word] = 1
        word_id = 0
        
        for per_word, per_count in word_freq.items():
            if per_count < self.min_count:  
                self.word_count_sum -= per_count
                continue
            self.id2word_dict[word_id] = per_word
            self.word2id_dict[per_word] = word_id
            self.wordId_frequency_dict[word_id] = per_count
            word_id += 1
        self.word_count = len(self.word2id_dict)

    def _init_sample_table(self):
        sample_table_size = 1e8
        pow_frequency = np.array(list(self.wordId_frequency_dict.values())) ** 0.75  
        word_pow_sum = sum(pow_frequency)  
        ratio_array = pow_frequency / word_pow_sum  
        word_count_list = np.round(ratio_array * sample_table_size)
        for word_index, word_freq in enumerate(word_count_list):
            self.sample_table += [word_index] * int(word_freq)  
        self.sample_table = np.array(self.sample_table)

    
    def get_batch_pairs(self, batch_size, window_size):
        while len(self.word_pairs_queue) < batch_size:
            for _ in range(10000):  
                
                sentence = self.input_file
                if sentence is None or sentence == '':
                    continue
                wordId_list = []  
                for words in sentence:
                    words = words.strip().split(' ')
                    for word in words:
                        try:
                            word_id = self.word2id_dict[word]
                            wordId_list.append(word_id)
                        except:
                            continue

                for i, wordId_w in enumerate(wordId_list):
                    for j, wordId_v in enumerate(wordId_list[max(i - window_size, 0):i + window_size + 1]):
                        assert wordId_w < self.word_count
                        assert wordId_v < self.word_count
                        if i == j:  
                            continue
                        self.word_pairs_queue.append((wordId_w, wordId_v))
        result_pairs = []  
        for _ in range(batch_size):
            result_pairs.append(self.word_pairs_queue.popleft())
        return result_pairs

    def get_pairs(self, pos_pairs):
        neg_word_pair = []
        pos_word_pair = []
        for pair in pos_pairs:
            pos_word_pair += zip([pair[0]] * len(self.huffman_pos_path[pair[1]]), self.huffman_pos_path[pair[1]])
            neg_word_pair += zip([pair[0]] * len(self.huffman_neg_path[pair[1]]), self.huffman_neg_path[pair[1]])
        return pos_word_pair, neg_word_pair

    def get_negative_sampling(self, positive_pairs, neg_count):
        neg_u = np.random.choice(self.sample_table, size=(len(positive_pairs), neg_count)).tolist()
        return neg_u



    def evaluate_pairs_count(self, window_size):
        return self.word_count_sum * (2 * window_size - 1) - (self.sentence_count - 1) * (1 + window_size) * window_size


In [ ]:
def test():
    test_data = InputData(raw_text, 1)
    test_data.evaluate_pairs_count(2)
    pos_pairs = test_data.get_batch_pairs(10, 2)
    
    pos_word_pairs = []
    for pair in pos_pairs:
        pos_word_pairs.append((test_data.id2word_dict[pair[0]], test_data.id2word_dict[pair[1]]))

    neg_pair = test_data.get_negative_sampling(pos_pairs, 2)

    neg_word_pair = []
    for pair in neg_pair:
        neg_word_pair.append((test_data.id2word_dict[pair[0]], test_data.id2word_dict[pair[1]]))

    print()
    print("=====================================")
    print()
    return test_data.get_pairs(pos_pairs), pos_pairs

In [ ]:
test()

Word Count is: 14
Word Count Sum is 14
Sentence Count is: 1
Word Frequency: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1}




(([(0, 24),
   (0, 15),
   (1, 14),
   (1, 24),
   (1, 15),
   (1, 24),
   (2, 14),
   (2, 24),
   (2, 24),
   (2, 23),
   (2, 16)],
  [(0, 26),
   (0, 24),
   (0, 14),
   (0, 26),
   (0, 23),
   (1, 26),
   (1, 24),
   (1, 26),
   (1, 23),
   (1, 26),
   (1, 23),
   (1, 15),
   (2, 26),
   (2, 24),
   (2, 26),
   (2, 24),
   (2, 14),
   (2, 26),
   (2, 23),
   (2, 15),
   (2, 26),
   (3, 26),
   (3, 24),
   (3, 14)]),
 [(0, 1),
  (0, 2),
  (1, 0),
  (1, 2),
  (1, 3),
  (2, 0),
  (2, 1),
  (2, 3),
  (2, 4),
  (3, 1)])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SkipGramModel(nn.Module):
    def __init__(self, emb_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.w_embeddings = nn.Embedding(2*emb_size-1, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(2*emb_size-1, emb_dimension, sparse=True)
        self._init_emb()

    def _init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.w_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)

    def forward(self, pos_w, pos_v,neg_w, neg_v):
        
        emb_w = self.w_embeddings(torch.LongTensor(pos_w))  
        neg_emb_w = self.w_embeddings(torch.LongTensor(neg_w))
        emb_v = self.v_embeddings(torch.LongTensor(pos_v))
        neg_emb_v = self.v_embeddings(torch.LongTensor(neg_v))  

        score = torch.mul(emb_w, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(-1 * score)
        neg_score = torch.mul(neg_emb_w, neg_emb_v).squeeze()
        neg_score = torch.sum(neg_score, dim=1)
        neg_score = F.logsigmoid(neg_score)
        
        loss = -1 * (torch.sum(score) + torch.sum(neg_score))
        return loss

    def save_embedding(self, id2word, file_name):
        embedding = self.w_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))

In [ ]:
import torch.optim as optim
from tqdm import tqdm


WINDOW_SIZE = 4  
BATCH_SIZE = 64  
MIN_COUNT = 3  
EMB_DIMENSION = 100  
LR = 0.02  
NEG_COUNT = 4 


class Word2Vec:
    def __init__(self, input_file_name, output_file_name):
        self.output_file_name = output_file_name
        self.data = InputData(input_file_name, MIN_COUNT)
        self.model = SkipGramModel(self.data.word_count, EMB_DIMENSION)
        self.lr = LR
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)

    def train(self):
        print("SkipGram Training......")
        pairs_count = self.data.evaluate_pairs_count(WINDOW_SIZE)
        print("pairs_count", pairs_count)
        batch_count = pairs_count / BATCH_SIZE
        print("batch_count", batch_count)
        process_bar = tqdm(range(int(batch_count)))
        for i in process_bar:
            pos_pairs = self.data.get_batch_pairs(BATCH_SIZE, WINDOW_SIZE)
            pos_pairs,neg_pairs = self.data.get_pairs(pos_pairs)
            pos_u = [pair[0] for pair in pos_pairs]
            pos_v = [int(pair[1]) for pair in pos_pairs]
            neg_u = [pair[0] for pair in neg_pairs]
            neg_v = [int(pair[1]) for pair in neg_pairs]
            self.optimizer.zero_grad()
            loss = self.model.forward(pos_u, pos_v, neg_u,neg_v)
            loss.backward()
            self.optimizer.step()

            if i * BATCH_SIZE % 100000 == 0:
                self.lr = self.lr * (1.0 - 1.0 * i / batch_count)
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = self.lr

        self.model.save_embedding(self.data.id2word_dict, self.output_file_name)

In [ ]:
w2v = Word2Vec(input_file_name=raw_text, output_file_name="word_embedding.txt")
w2v.train()

0it [00:00, ?it/s]

Word Count is: 4
Word Count Sum is 16
Sentence Count is: 5
SkipGram Training......
pairs_count 32
batch_count 0.5
